In [2]:
import numpy as np
import os
import glob
import pandas as pd
from statsmodels.sandbox.stats.multicomp import multipletests
import numpy as np
from subprocess import call

In [3]:
dir_pre = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/apa/nano_merge"

output_path = f"{dir_pre}_signal_3aqtl.csv"
output_all_path = f"{dir_pre}_polyA_SNP.csv"

In [4]:
files = glob.glob(f'{dir_pre}_haplotype_chr*_tmp.csv')
dfs = [pd.read_csv(file) for file in files]
df = pd.concat(dfs, ignore_index=True)
df = df.sort_values(by=['chrom', 'snp_pos_1base'])
df = df.reset_index(drop=True)

In [6]:
p_adjusted = multipletests(df["p_value"], method='fdr_bh')
df['fdr'] = p_adjusted[1]
print("pvalue小于0.05的个数：", len(df[df['p_value']<0.05]))
print("pvalue小于0.01的个数：", len(df[df['p_value']<0.01]))
print("fdr小于0.1的个数：", len(df[df['fdr']<0.1]))

pvalue小于0.05的个数： 561
pvalue小于0.01的个数： 225
fdr小于0.1的个数： 106


In [7]:
result_df = df[df['p_value']<0.05]
result_df.to_csv(output_path, index=False)
df.to_csv(output_all_path, index=False)
call(f'rm {dir_pre}_haplotype_chr*_tmp.csv',shell=True)

0

In [17]:
from collections import Counter
import pandas as pd

def analyze_snp_apa(dict1, dict2):
    # 步骤1：根据SNP将read分类
    ref_reads = set()
    alt_reads = set()
    for read_id, base in dict1.items():
        if base == 'ref':
            ref_reads.add(read_id)
        elif base == 'alt':
            alt_reads.add(read_id)

    # 步骤2：统计每类read的APA分类
    ref_apa = Counter(dict2[read_id] for read_id in ref_reads if read_id in dict2)
    alt_apa = Counter(dict2[read_id] for read_id in alt_reads if read_id in dict2)

    # 步骤3：创建多维表
    all_apa_types = set(ref_apa.keys()) | set(alt_apa.keys())
    data = {
        'ref': [ref_apa.get(apa, 0) for apa in all_apa_types],
        'alt': [alt_apa.get(apa, 0) for apa in all_apa_types]
    }
    df = pd.DataFrame(data, index=list(all_apa_types))
    print(df)

    # 步骤4：找出总数最多的APA分类，并创建四格表
    total_counts = df.sum(axis=1)
    top_apa = total_counts.idxmax()
    
    new_df = pd.DataFrame({
        'ref': [df.loc[top_apa, 'ref'], df.drop(top_apa).sum()['ref']],
        'alt': [df.loc[top_apa, 'alt'], df.drop(top_apa).sum()['alt']]
    }, index=[top_apa, 'others'])
    
    return new_df

# 使用示例
dict1 = {'read1': 'ref', 'read2': 'alt', 'read3': 'ref', 'read4': 'alt', 'read5': 'ref'}
dict2 = {'read1': 'APA1', 'read2': 'APA2', 'read3': 'APA1', 'read4': 'APA3', 'read5': 'APA4'}

result = analyze_snp_apa(dict1, dict2)
print(result)

      ref  alt
APA2    0    1
APA4    1    0
APA1    2    0
APA3    0    1
        ref  alt
APA1      2    0
others    1    2


In [19]:
result.index.tolist()

['APA1', 'others']